<a href="https://colab.research.google.com/github/gabrielborja/python_data_analysis/blob/main/abonnent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Abonnenter

## Loading packages

In [1]:
# Importing necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from ipywidgets import interact
import plotly.express as px

In [2]:
# Setting theme for visuals
sns.set_theme(rc = {'figure.figsize': (8, 4)})

In [ ]:
# Unzipping data
!unzip data.zip

In [ ]:
# Reading data and saving to dataframe
df = pd.read_csv("dataset_ice.csv", sep="|")

In [6]:
df.iloc[21:24]

,DATA_SOURCE,PRICEPLAN_ID,SYSTEM_NAME,EVENT_TYPE,EVENT_SUBTYPE,ACTIVATION_TYPE,DATE,H_MAIN_ID,H_DATA_ID,INCLUDED_MB,LIFETIME_DAYS,FEE,DISCOUNT,OPERATOR,CUSTOMER_SEGMENT,PRODUCT_FAMILY,PRODUCT_GENERATION,PRODUCT_TYPE,PRODUCT_NAME,MARKET_SEGMENT,MARKET_SEGMENT_GROUP,SALES_CHANNEL_NAME,SALES_CHANNEL_CHAIN_NAME,SALES_CHANNEL_DEALER_NAME,PRICEPLAN_FULL_MONTH,RGU_FULL_MONTH,NATIONAL_MB,INTROAM_INSIDE_EU_EEC_MB,INTROAM_OUTSIDE_EU_EEC_MB
21,Activation,INB-11876696-0,INB,Addition,NaN,Addition,2018-01-12 00.00.00.000000,c2d5a99c1dfeb6ad39c665151072e516,NaN,6144,NaN,349.0,0.0,833.0,Business,Smartphone,PPG2,Bedrift 6 GB,Bedrift 6 GB,SOHO,SOHO,Outbound,Instacall,Instacall AS,NaN,NaN,NaN,NaN,NaN
22,Activation,INB-11909257-0,INB,Addition,NaN,Addition,2018-01-15 00.00.00.000000,7b0af7bf3dd978f5ba137aba9a8899ba,NaN,1024,NaN,129.0,0.0,815.0,Business,Smartphone,PPG2,Bedrift 1 GB,Bedrift 1 GB,SOHO,SOHO,Outbound,Instacall,Instacall AS,NaN,NaN,NaN,NaN,NaN
23,Activation,INB-11909272-0,INB,Addition,NaN,Addition,2018-01-15 00.00.00.000000,c4a78eb381580fa9d1f4e17b1ec5af77,NaN,1024,NaN,129.0,0.0,815.0,Business,Smartphone,PPG2,Bedrift 1 GB,Bedrift 1 GB,SOHO,SOHO,Outbound,Instacall,Instacall AS,NaN,NaN,NaN,NaN,NaN


## Data Manipulation

In [4]:
# Applying regex to convert string to valid date format
df['DATE'].replace(to_replace=r'( 00\.00\.)', value=' 00:00:', regex=True, inplace=True)

In [5]:
# Converting string to datetime column
df['DATE'] = pd.to_datetime(df['DATE'], format='%Y-%m-%d %H:%M:%S')

In [6]:
# Slicing dataset to use only selected columns
col_slice = ['DATE', 'DATA_SOURCE', 'PRICEPLAN_ID', 'INCLUDED_MB', 'LIFETIME_DAYS', 'FEE', 'DISCOUNT', 'OPERATOR',
             'PRODUCT_GENERATION', 'PRODUCT_TYPE', 'PRODUCT_NAME', 'MARKET_SEGMENT', 'SALES_CHANNEL_NAME',
             'SALES_CHANNEL_CHAIN_NAME', 'SALES_CHANNEL_DEALER_NAME', 'PRICEPLAN_FULL_MONTH', 'RGU_FULL_MONTH']
df = df[col_slice]

In [53]:
# Assigning categories from datetime
df = df.assign(Månd_num = df['DATE'].dt.month,
               Månd = df['DATE'].dt.month_name().str[:3])

In [54]:
# Checking the dataframe info
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 495818 entries, 0 to 495817
Data columns (total 19 columns):
 #   Column                     Non-Null Count   Dtype         
---  ------                     --------------   -----         
 0   DATE                       495818 non-null  datetime64[ns]
 1   DATA_SOURCE                495818 non-null  object        
 2   PRICEPLAN_ID               495818 non-null  object        
 3   INCLUDED_MB                495818 non-null  int64         
 4   LIFETIME_DAYS              477033 non-null  float64       
 5   FEE                        495712 non-null  float64       
 6   DISCOUNT                   495712 non-null  float64       
 7   OPERATOR                   28073 non-null   float64       
 8   PRODUCT_GENERATION         495818 non-null  object        
 9   PRODUCT_TYPE               495818 non-null  object        
 10  PRODUCT_NAME               495818 non-null  object        
 11  MARKET_SEGMENT             492956 non-null  object  

In [8]:
# Creating a list of the columns
col_list = list(df.columns)

In [9]:
# Analyzing the value proportion for each column
@interact(Col=col_list)
def proportion_count(Col):
  #return df.loc[:, Col].value_counts()
  return df.value_counts(subset=Col, normalize=True, dropna=False).reset_index(name="Proportion")

interactive(children=(Dropdown(description='Col', options=('DATE', 'DATA_SOURCE', 'PRICEPLAN_ID', 'INCLUDED_MB…

## Q1

In [12]:
# Aggregating subscribers data by market segment
df1_abo_a = df[df['DATA_SOURCE']=='Stock'].groupby(by=['DATE', 'MARKET_SEGMENT']).agg(Abonnenter = ('PRICEPLAN_ID', 'count')).reset_index()

In [74]:
# Plotting the subscribers by market segment
fig1_a = px.bar(data_frame=df1_abo_a, x='DATE', y='Abonnenter', color='MARKET_SEGMENT', height=450, width=800)
fig1_a.update_layout(title_text='Abonnenter i 2018 - Markedssegment', title_x=0.5)
fig1_a.show()

In [15]:
# 1. Hvordan har abonnementsutviklingen (stock) vært gjennom 2018?
df1_abo_b = df1_abo_a.groupby(by=['DATE']).agg(Abonnenter = ('Abonnenter', 'sum')).reset_index()
df1_abo_b

,DATE,Abonnenter
0,2018-01-31,35821
1,2018-02-28,36677
2,2018-03-31,37293
3,2018-04-30,38254
4,2018-05-31,38575
5,2018-06-30,38937
6,2018-07-31,39391
7,2018-08-31,39466
8,2018-09-30,39066
9,2018-10-31,38740


In [30]:
# Plotting subscribers data trend
fig1_b = px.line(data_frame=df1_abo_b, x='DATE', y='Abonnenter', height=450, width=800, text='Abonnenter')
fig1_b.update_layout(title_text='Abonnenter i 2018', title_x=0.5)
fig1_b.update_traces(textposition="top center")
fig1_b.show()

## Q2

In [17]:
# Aggregating the activations by sales channel
df2_sal_a = df[df['DATA_SOURCE']=='Activation'].groupby(by=['DATE', 'SALES_CHANNEL_NAME']).agg(Aktiveringer = ('PRICEPLAN_ID', 'count')).reset_index()
df2_sal_a = df2_sal_a.assign(Månd_num = df2_sal_a['DATE'].dt.month, Månd = df2_sal_a['DATE'].dt.month_name().str[:3])
df2_sal_a = df2_sal_a.groupby(by=['Månd_num', 'Månd', 'SALES_CHANNEL_NAME']).agg(Aktiveringer = ('Aktiveringer', 'sum')).reset_index()
df2_sal_a

,Månd_num,Månd,SALES_CHANNEL_NAME,Aktiveringer
0,1,Jan,Ice Sales Partner,137
1,1,Jan,Inbound,179
2,1,Jan,Internal,8
3,1,Jan,Mobile Data Sales,15
4,1,Jan,Outbound,709
...,...,...,...,...
80,12,Dec,Ice Sales Partner,263
81,12,Dec,Inbound,188
82,12,Dec,Outbound,438
83,12,Dec,Partner Sales,236


In [75]:
# Plotting the activations by sales channel
fig2_a = px.bar(data_frame=df2_sal_a, x='Månd', y='Aktiveringer', color='SALES_CHANNEL_NAME', height=450, width=800)
fig2_a.update_layout(title_text='Aktiveringer i 2018 - Salgskanal', title_x=0.5)
fig2_a.show()

In [19]:
# 2. Hvordan har salgsutviklingen (aktiveringer) vært gjennom 2018?
df2_sal_b = df2_sal_a.groupby(by=['Månd_num', 'Månd']).agg(Aktiveringer = ('Aktiveringer', 'sum')).reset_index()
df2_sal_b

,Månd_num,Månd,Aktiveringer
0,1,Jan,1564
1,2,Feb,1705
2,3,Mar,1680
3,4,Apr,1851
4,5,May,1656
5,6,Jun,1824
6,7,Jul,1196
7,8,Aug,1241
8,9,Sep,1072
9,10,Oct,1309


In [21]:
# Plotting the activation over the year
fig2_b = px.bar(data_frame=df2_sal_b, x='Månd', y='Aktiveringer', height=450, width=800, text_auto=True)
fig2_b.update_layout(title_text='Aktiveringer i 2018', title_x=0.5)
fig2_b.show()

## Q3

In [45]:
# Aggregating the cancellations by market segment
df3_a = df[df['DATA_SOURCE']=='Cancellation'].groupby(by=['DATE', 'MARKET_SEGMENT']).agg(Kanselleringer = ('PRICEPLAN_ID', 'count')).reset_index()
df3_a = df3_a.assign(Månd_num = df3_a['DATE'].dt.month, Månd = df3_a['DATE'].dt.month_name().str[:3])
df3_a = df3_a.groupby(by=['Månd_num', 'Månd', 'MARKET_SEGMENT']).agg(Kanselleringer = ('Kanselleringer', 'sum')).reset_index()
df3_a.tail()

,Månd_num,Månd,MARKET_SEGMENT,Kanselleringer
55,12,Dec,ENTERPRISE,176
56,12,Dec,MEDIUM BUS,352
57,12,Dec,PUBLIC,1
58,12,Dec,SMALL BUS,208
59,12,Dec,SOHO,651


In [73]:
# Plotting the cancellations by market segment
fig3_a = px.bar(data_frame=df3_a, x='Månd', y='Kanselleringer', color='MARKET_SEGMENT', height=450, width=800)
fig3_a.update_layout(title_text='Kanselleringer i 2018 - Markedssegment', title_x=0.5)
fig3_a.show()

In [47]:
# 3. Hvordan har churnutviklingen vært gjennom 2018?
df3_b = df3_a.groupby(by=['Månd_num', 'Månd']).agg(Kanselleringer = ('Kanselleringer', 'sum')).reset_index()
df3_b

,Månd_num,Månd,Kanselleringer
0,1,Jan,1358
1,2,Feb,927
2,3,Mar,1140
3,4,Apr,965
4,5,May,1389
5,6,Jun,1473
6,7,Jul,753
7,8,Aug,1198
8,9,Sep,1537
9,10,Oct,1688


In [48]:
# Plotting the cancellations over the year
fig3_b = px.bar(data_frame=df3_b, x='Månd', y='Kanselleringer', height=450, width=800, text_auto=True)
fig3_b.update_layout(title_text='Kanselleringer i 2018', title_x=0.5)
fig3_b.show()

## Q4

In [60]:
# 4. Til hvilken konkurrent mister ice flest abonnenter?
df4_a = df[df['DATA_SOURCE']=='Cancellation'].groupby(by=['OPERATOR']).agg(Kansellering = ('PRICEPLAN_ID', 'count')).reset_index()
df4_a

,OPERATOR,Kansellering
0,702.0,53
1,705.0,498
2,725.0,2
3,736.0,31
4,739.0,6
5,743.0,2
6,809.0,4
7,815.0,3158
8,817.0,26
9,819.0,799


In [72]:
# Plotting the cancellations by competitor
fig4_a = px.treemap(data_frame=df4_a, path=['OPERATOR'], values='Kansellering', height=450, width=800)
fig4_a.update_layout(title_text='Kanselleringer i 2018 - Konkurrent', title_x=0.5)
fig4_a.show()

## Q5

In [ ]:
# 5. Fra hvilken konkurrent vinner ice flest abonnenter?
df5_akt = df[df['DATA_SOURCE']=='Activation'].groupby(by=['DATE', 'DATA_SOURCE', 'OPERATOR']).agg(COUNT = ('PRICEPLAN_ID', 'count')).reset_index()
df5_akt['OPERATOR'] = df5_akt['OPERATOR'].astype(str)
df5_akt

In [ ]:
# Checking the value counts
df5_akt.value_counts(subset='OPERATOR', normalize=True).reset_index(name='NYE ABONNENTER')

In [ ]:
# Plotting the Count Q5 over the year
sns.scatterplot(x='DATE', y='COUNT', data=df5_akt[df5_akt['OPERATOR'].isin(['832.0', '815.0', '705.0'])], hue='OPERATOR')
plt.title('Count over the year')
plt.show()

In [ ]:
#6. Hvor stor markedsandel hadde ice ved utgangen av 2018?


In [ ]:
df[df['PRICEPLAN_ID'].isin(['INB-9918193-0'])]

In [ ]:
# KRONER - #7a Hva mener du er fire viktige innsikter i datasettet uavhengig av de konkrete spørsmålene over?
df7_a = df.groupby(by=['DATE']).agg(KRONER = ('FEE', 'sum')).reset_index()
df7_a = df7_a.assign(MONTH = df7_a['DATE'].dt.month,
                     WEEKNUM = df7_a['DATE'].dt.isocalendar().week)
df7_a = df7_a.groupby(by=['MONTH']).agg(KRONER = ('KRONER', 'sum')).reset_index()
df7_a

In [ ]:
# Plotting the Count Q7a over the year
sns.lineplot(x='MONTH', y='KRONER', data=df7_a)
plt.title('Sum over the year')
plt.show()

In [ ]:
# DISCOUNT - #7b Hva mener du er fire viktige innsikter i datasettet uavhengig av de konkrete spørsmålene over?
df7_b = df.groupby(by=['DATE']).agg(DISCOUNT = ('DISCOUNT', 'sum')).reset_index()
df7_b = df7_b.assign(MONTH = df7_b['DATE'].dt.month,
                     WEEKNUM = df7_b['DATE'].dt.isocalendar().week)
df7_b = df7_b.groupby(by=['MONTH']).agg(DISCOUNT = ('DISCOUNT', 'sum')).reset_index()
df7_b = df7_b.assign(DISCOUNT = df7_b['DISCOUNT']*-1)
df7_b

In [ ]:
# Plotting the Count Q7a over the year
sns.lineplot(x='MONTH', y='DISCOUNT', data=df7_b)
plt.title('Sum over the year')
plt.show()

In [ ]:
# TIME_DAYS - #7c Hva mener du er fire viktige innsikter i datasettet uavhengig av de konkrete spørsmålene over?
sns.histplot(x='LIFETIME_DAYS', data=df[df['DATA_SOURCE']=='Stock'], bins=11, stat='percent')
plt.title('Histogram av LIFETIME_DAYS')
plt.show()

In [ ]:
# MEGAS - #7d Hva mener du er fire viktige innsikter i datasettet uavhengig av de konkrete spørsmålene over?
df7_d = df.value_counts(subset='INCLUDED_MB', normalize=True, dropna=False).reset_index(name='Proportion')
df7_d['INCLUDED_MB'].astype('category')
df7_d

In [ ]:
# Plotting the Q7d
sns.barplot(x='INCLUDED_MB', y='Proportion', data=df7_d)
plt.title('Popular Subscription plans')
plt.show()